# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symmer.symplectic import PauliwordOp, QuantumState

Firstly, note the correct phases under multiplication by a Pauli Y:

In [2]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp.from_list(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

 1.000+0.000j |0> ->  0.000+1.000j |1>
 1.000+0.000j |1> ->  0.000-1.000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [3]:
psi = QuantumState([[0,0]])
had = PauliwordOp.from_list(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state:  1.000+0.000j |00>

After application of the Hadamard gate:

 0.500+0.000j |00> +
 0.500+0.000j |01> +
 0.500+0.000j |10> +
 0.500-0.000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [4]:
print(eq_superposition.state_op)

 0.500+0.000j ZZ +
 0.500+0.000j ZX +
 0.500+0.000j XZ +
 0.500-0.000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [5]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [6]:
N = 10 # number of qubits
M = 10 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

 0.263+0.182j |1101010011> +
 0.287+0.149j |1110011011> +
 0.047+0.315j |1001011111> +
 0.337+0.140j |0010011101> +
 0.210+0.130j |0011111100> +
 0.133+0.343j |1111110000> +
 0.188+0.049j |0111010101> +
 0.195+0.221j |1011000000> +
 0.276+0.189j |1011010000> +
 0.334+0.119j |1001001101>

 0.164+0.343j |1010111101> +
 0.024+0.171j |1110110011> +
 0.126+0.193j |1000111010> +
 0.273+0.247j |0111100000> +
 0.180+0.323j |1010110100> +
 0.094+0.271j |0001101111> +
 0.317+0.170j |1111111001> +
 0.295+0.253j |0111100000> +
 0.141+0.101j |1111100100> +
 0.272+0.181j |1010011101>


In [7]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.conjugate * psi)

 0.062+0.179j |0001101111> +
 0.223+0.093j |0010011101> +
 0.139+0.086j |0011111100> +
 0.125+0.032j |0111010101> +
 0.376+0.330j |0111100000> +
 0.083+0.128j |1000111010> +
 0.221+0.079j |1001001101> +
 0.031+0.209j |1001011111> +
 0.180+0.120j |1010011101> +
 0.119+0.214j |1010110100> +
 0.108+0.227j |1010111101> +
 0.129+0.146j |1011000000> +
 0.182+0.125j |1011010000> +
 0.174+0.120j |1101010011> +
 0.190+0.098j |1110011011> +
 0.016+0.113j |1110110011> +
 0.093+0.067j |1111100100> +
 0.088+0.227j |1111110000> +
 0.210+0.112j |1111111001>

Norm: (1.0000000000000002+0j)


In [8]:
op = random_herm_op(N, 10)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.conjugate * op * psi)

 5.125+0.000j ZIIYYYIYXX +
-0.120+0.000j YYXXYXYZXZ +
 1.356+0.000j YYXXYXYZXY +
 1.144+0.000j IIZYXXIXYY +
 0.250+0.000j XXIZYXYZXY +
-3.997+0.000j YXYYZZXIZY +
 0.874+0.000j ZXYYYZZYIZ +
 0.737+0.000j IIIYZXIXYZ +
 8.325+0.000j YXYXZIIXXX +
-0.195+0.000j YYXZZYYIIY

Expectation value <psi|op|psi> = (-0.3411776054252657+1.3877787807814457e-17j)


It is also possible to convert from a $2^N$ dimensional statevector over $N$ qubits to a `QuantumState` object, via the `array_to_QuantumState` function:

In [9]:
statevector = np.zeros(2**10).reshape([-1,1])
statevector[0]   = np.sqrt(1/6)
statevector[250] = np.sqrt(2/6)
statevector[500] = np.sqrt(3/6)
psi = QuantumState.from_array(statevector)
print(psi)
print(psi.conjugate * psi)

 0.408+0.000j |0000000000> +
 0.577+0.000j |0011111010> +
 0.707+0.000j |0111110100>
(1+0j)
